In [61]:
from collections import Counter


cnt = Counter()
cntpositivas = Counter()
cntnegativas = Counter()
diffpositivas = []
diffnegativas = []
cadena = ""
#Limpieza de datos
for x in archivo:
    x = x.replace(".","")
    x = x.replace(",","")
    x = x.replace("\n","")
    cadena += x+" "

cadena = cadena.upper()
#print(cadena)


cadena = cadena.split()
#print(cadena)
for x in cadena:
    cnt[x]+=1

vocabulario = len(cnt)
print(f"Vocabulario: {vocabulario}")


#POSITIVOS 
positivas = 'I loved the movie A great movie Good movie Great acting A good movie'
positivas = positivas.upper()
positivas = positivas.split()

for x in positivas:
    cntpositivas[x]+=1   
    
for i in cntpositivas.keys():
    diffpositivas.append(i)
    
totalpositivas = len(positivas)

#NEGATIVO
negativas = 'I hated the movie Poor acting'
negativas = negativas.upper()
negativas = negativas.split()

for x in negativas:
    cntnegativas[x]+=1

for i in cntnegativas.keys():
    diffnegativas.append(i)


totalnegativas = len(negativas)

print(f"Total palabras positivas: {totalpositivas}")
print(f"Total palabras negativas: {totalnegativas}")

Ppositiva = 3/5
Pnegativa = 2/5

print(f"P +: {Ppositiva}")
print(f"P -: {Pnegativa}")

Vocabulario: 10
Total palabras positivas: 14
Total palabras negativas: 6
P +: 0.6
P -: 0.4


In [85]:
dpositivas = {}
dnegativas = {}

for x in diffpositivas:
    dpositivas["{0}".format(x)] = ((positivas.count(x)+1)/(totalpositivas+vocabulario))
    
for j in diffnegativas:
    dnegativas["{0}".format(j)] = ((negativas.count(x)+1)/(totalnegativas+vocabulario))

#print("DICCIONARIO CON EL PROBABILIDAD DE CADA PALABRA")
#print(dpositivas)
#print(dnegativas)
    
print("DOCUMENTO NUEVO")
print(documentoNuevo)

documentoNuevo = "I hated the poor acting"
documentoNuevo = documentoNuevo.upper()
documentoNuevo = documentoNuevo.split()

calculoPositivo = Ppositiva
for x in documentoNuevo:
    if x in dpositivas:
        calculoPositivo *= dpositivas[x]
    else:
        calculoPositivo *= ((0+1)/(totalpositivas+vocabulario))
        
print(f"Proporcion del que el documento sea de la clase positiva {calculoPositivo}")


calculoNegativa = Pnegativa
for x in documentoNuevo:
    if x in dnegativas:
        calculoNegativa *= dnegativas[x]
    else:
        calculoNegativa *= ((0+1)/(totalnegativas+vocabulario))

print(f"Proporcion del que el documento sea de la clase negativo {calculoNegativa}")        


if calculoPositivo > calculoNegativa:
    print("Este documento tiene mas proporcion de la clase positivo")
elif calculoPositivo < calculoNegativa:
    print("Este documento tiene mas proporcion de la clase negativa")
elif calculoPositivo == calculoNegativa:
    print("Este documento tiene la misma proporcion en las dos clases")

    
    

DOCUMENTO NUEVO
['I', 'HATED', 'THE', 'POOR', 'ACTING']
Proporcion del que el documento sea de la clase positiva 6.028163580246911e-07
Proporcion del que el documento sea de la clase negativo 1.220703125e-05
Este documento tiene mas proporcion de la clase negativa
